In [296]:
def get_server_list(conf_file):
    with open(conf_file,'r') as f:
        s = f.read()
    print(s)

    import re
    rs = {}
    for m in [m for m in re.finditer('\[([^]]+)\]([^\[]+)', re.sub('\s*#.*','', s) )]:
        rs[m.group(1)] = [re.sub('\s*#.*','', l).strip() for l in m.group(2).strip().splitlines()]
    return rs

conf_file = "/data/batch_ssh_servers.conf"
conf = get_server_list(conf_file)
print(conf)
g = conf['batch1']
g

[batch1]
# comments here
us # 1st
notworking # test except
hk # 3rd

[batch2]
# 2nd section

[batch 3]

{'batch1': ['us', 'notworking', 'hk'], 'batch2': [], 'batch 3': []}


['us', 'notworking', 'hk']

In [303]:
connect_kwargs={
    "key_filename": "/data/id_rsa",
}
    
from fabric import Connection

import fabric
fabric.ThreadingGroup

conn_list = [ Connection(h, port=10220,  connect_kwargs=connect_kwargs) for h in g ]

from fabric import ThreadingGroup as Group
pool= Group.from_connections(conn_list)
pool

[<Connection host=us port=10220>,
 <Connection host=notworking port=10220>,
 <Connection host=hk port=10220>]

In [362]:
from fabric.exceptions import GroupException

def append_GroupResult(GroupResult_list, pool, cmd_list, **kwargs):
    if len(pool) == 0: return GroupResult_list
    
    if type(cmd_list) == str:
        cmd_list = cmd_list.strip().splitlines()
        
    for cmd in cmd_list:
        try:
            GroupResult_list.append(pool.run(cmd, **kwargs))
        except GroupException as e:
            GroupResult_list.append(e.result)
            
    return GroupResult_list


def group_run(pool, cmd_list, **kwargs):    
    GroupResult_list = append_GroupResult([], pool, 'hostname')
    #pool_work = Group.from_connections([c for c in pool if type(GroupResult_list[0].get(c)) == fabric.runners.Result])
    pool_work = Group.from_connections([c for c in pool if c.is_connected])    
    return append_GroupResult(GroupResult_list, pool_work, cmd_list, **kwargs)

cmds="""
date
whoami
ls /etc/hosts*
"""
run_rs = group_run(pool,cmds,hide='out')
run_rs

HK1C1GTX
bwg2c2g


[{<Connection host=hk port=10220>: <Result cmd='hostname' exited=0>,
  <Connection host=us port=10220>: <Result cmd='hostname' exited=0>,
  <Connection host=notworking port=10220>: socket.gaierror(-2,
                  'Name or service not known')},
 {<Connection host=hk port=10220>: <Result cmd='date' exited=0>,
  <Connection host=us port=10220>: <Result cmd='date' exited=0>},
 {<Connection host=hk port=10220>: <Result cmd='whoami' exited=0>,
  <Connection host=us port=10220>: <Result cmd='whoami' exited=0>},
 {<Connection host=hk port=10220>: <Result cmd='ls /etc/hosts*' exited=0>,
  <Connection host=us port=10220>: <Result cmd='ls /etc/hosts*' exited=0>}]

In [368]:
rs

{<Connection host=hk port=10220>: <Result cmd='ls /etc/hosts*' exited=0>,
 <Connection host=us port=10220>: <Result cmd='ls /etc/hosts*' exited=0>}

In [378]:
def run_rs_to_dict_list(run_rs, splitlines=True):
    run_rs_list = []
    for rs in run_rs:
        for k,v in rs.items():
            item = {'Host': k.host}
            if type(v) == fabric.runners.Result:
                d = vars(v)
                for k in ['command','exited']: 
                    item[k] = d[k]
                for k in ['stdout','stderr']: 
                    if splitlines: item[k] = d[k].strip().splitlines()
                    else:          item[k] = d[k].strip()
            else:
                item['ConnError'] = str(v)

            run_rs_list.append(item)

    return run_rs_list
        
import json

with open("fabric2_result.json", "w") as f:
    json.dump(run_rs_to_dict_list(run_rs), f, indent=4)        

In [379]:
!cat fabric2_result.json

[
    {
        "Host": "hk",
        "command": "hostname",
        "exited": 0,
        "stdout": [
            "HK1C1GTX"
        ],
        "stderr": []
    },
    {
        "Host": "us",
        "command": "hostname",
        "exited": 0,
        "stdout": [
            "bwg2c2g"
        ],
        "stderr": []
    },
    {
        "Host": "notworking",
        "ConnError": "[Errno -2] Name or service not known"
    },
    {
        "Host": "hk",
        "command": "date",
        "exited": 0,
        "stdout": [
            "Wed May  1 18:08:54 CST 2019"
        ],
        "stderr": []
    },
    {
        "Host": "us",
        "command": "date",
        "exited": 0,
        "stdout": [
            "Wed May  1 06:08:54 EDT 2019"
        ],
        "stderr": []
    },
    {
        "Host": "hk",
        "command": "whoami",
        "exited": 0,
        "stdout": [
            "root"
        ],
        "stderr": []
    },
    {
        "Host": "us",
        "command": "whoami",
   

In [429]:
import pandas as pd
df = pd.DataFrame(run_rs_to_dict_list(run_rs, splitlines=False)).fillna('')
df = df.set_index('Host', append=True).sort_index(level=1).reset_index(level=1)
df

,Host,ConnError,command,exited,stderr,stdout
0,hk,,hostname,0,,HK1C1GTX
3,hk,,date,0,,Wed May 1 18:08:54 CST 2019
5,hk,,whoami,0,,root
7,hk,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny
2,notworking,[Errno -2] Name or service not known,,,,
1,us,,hostname,0,,bwg2c2g
4,us,,date,0,,Wed May 1 06:08:54 EDT 2019
6,us,,whoami,0,,root
8,us,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny


In [430]:
df[df['command'] == 'whoami']

,Host,ConnError,command,exited,stderr,stdout
5,hk,,whoami,0,,root
6,us,,whoami,0,,root


In [431]:
df[df['command'].str.contains('ls', regex=False)]

,Host,ConnError,command,exited,stderr,stdout
7,hk,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny
8,us,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny


In [334]:
pool.close()

In [335]:
for c in pool:
    print(c.is_connected)

False
False
False
